In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error
from sklearn.cluster import KMeans
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# armamento del Data Frame proveniente de Scikit-Learn

data = datasets.fetch_california_housing()

df = pd.DataFrame(data["data"],columns=data["feature_names"])
df["MedHouseVal"] = data["target"]
df["MedHouseVal"] = df["MedHouseVal"] * 100000

df_model = df.copy() # generando una copia, asi no afectamos al Data Frame original

def remove_outliers(df,columns):
    for c in columns:
        df[c] = df[c].mask(zscore(df[c]).abs() > 3, np.nan)
    
    return df

# removiendo valores atípicos que afecten el desempeño del modelo
df_model = remove_outliers(df_model, df.columns)
df_model.dropna(inplace=True)

df.describe()

#### Random Search CV(Cross-Validation)
Es una función utilizada en la búsqueda de los hiperparámetros de un modelo de aprendizaje automático. Su proceso se basa en crear un cuadrícula de todos los posibles valores, se realizan diferentes combinaciones de hiperparámetros para generar una variabilidad de modelos y se seleccionrán aleatoriamente algunos de ellos sobre los cuales se pondrán a prueba mediante una técnica denominada Cross-Validation, que consiste en las particiones de conjuntos de datos en n cantidades iguales y en las que una será utlizada para evualar el rendimiento del modelo generado, repitiendo el proceso n veces. Por último, la función almacenará no solo las puntuaciones obtenidas sino también los parámetros cuyo desempeño fue mayor al resto.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df[df.columns[:-1]],
                                                    df["MedHouseVal"],
                                                    test_size=0.25)

# ajuste de hiperparámetros utilizando RandomizedSearchCV()

xgbr_test = XGBRegressor()           

turned_parameters = {
    "n_estimators":[100,200,300,400],
    "subsample":[0.7,0.75,0.8,0.85],
    "max_depth":[3,4,5,6],
    "learning_rate":[0.2,0.3,0.4,0.5],
    "min_child_weight":[2,3,4,5],
    "gamma":[0,1,2,3]
}

random_search = RandomizedSearchCV(xgbr_test, turned_parameters,cv=5)
random_search.fit(x_train, y_train)

print(f'''n_estimators(número de modelos creados): {random_search.best_params_["n_estimators"]}
subsample(tamaño de muestra requerida): {random_search.best_params_["subsample"]}
max_depth(máxima profundidad de cada árbol): {random_search.best_params_["max_depth"]}
learning_rate(tasa de aprendizaje, evita el sobreajuste): {random_search.best_params_["learning_rate"]}
min_child_weight(suma mínima de peso de instancia necesaria en un nodo): {random_search.best_params_["min_child_weight"]}
gamma(cuanto mayor sea, más conservador será el modelo): {random_search.best_params_["gamma"]}''')

#### Boosting
En el ejemplo anterior habiamos mencionado y ejecutado la técnica de Bagging, donde generábamos distintos metaestimadores entrenados de forma independiente y se tomaban en cuenta los resultados de cada uno. En este caso, cada metaestimador capacitará y solucionará los errores del siguiente potenciando cada vez más la precisión final, es decir que su escalabilidad es vertical en lugar de horizontal. Requieren de una hiperparametrización compleja y una tendencia, si no realiza correctamente lo anterior mencionado, al sobreajuste, es decir, modelos con una precisión alta sobre un grupos de datos en particular pero con la poca adaptación para resolver nuevas situaciones.

In [ ]:
# asignando valores de los modelos que obtuvieron los mejores resultados

xgbr = XGBRegressor(n_estimators = random_search.best_params_["n_estimators"],
                    subsample = random_search.best_params_["subsample"],
                    max_depth = random_search.best_params_["max_depth"],
                    learning_rate = random_search.best_params_["learning_rate"],
                    min_child_weight = random_search.best_params_["min_child_weight"],
                    gamma = random_search.best_params_["gamma"])

xgbr.fit(x_train, y_train)

#### Métricas de regresión
Tienen mucha similitud con las medidas de variabilidad y pueden plantearse analogías entre estas, con la diferencia de que emplean los residuos del modelo para obtener los valores:

Error absoluto medio(MAE): similar a la desviacíon media absoluta, es la división por el número de muestra de la sumatoria de los residuos absolutos.

Error cuadrático medio(MSE): similar a la varianza, es la división por el número de muestra de la sumatoria de los residuos al cuadrado.

Raíz cuadrada del error cuadrático medio(RMSE): similar a la desviación éstandar, se utiliza para convertir a la unidad base de los valores al error cuadrático medio.

Coeficiente de determinación(R2 Score): valor de 0 a 1 que evalúa la bondad de ajuste de nuestro modelo a los datos, pero solo utilizada en modelos lineales.

In [ ]:
# regresión
predicts = xgbr.predict(x_test)

# validación con métricas de rendimiento
metrics = f"MAE: {round(mean_absolute_error(y_test,predicts))} | MSE: {round(mean_squared_error(predicts,y_test))} | RMSE: {round(root_mean_squared_error(predicts,y_test))}"
scatter = go.Figure()
scatter.add_trace(go.Scatter(x=predicts,y=y_test,mode="markers",marker_color="red"))
scatter.update_layout(title=metrics)
scatter

#### Dashboard que funciona como mapa descriptivo de las viviendas de California

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div(id="body",className="e5_body",children=[
    html.H1("Viviendas en California ",id="title",className="e5_title"),
        html.Div(id="div",className="e5_div",children=[
            dcc.Dropdown(id="dropdown",className="e5_dropdown",
                        options = [
                            {"label":"Valor de precio","value":"MedHouseVal"},
                            {"label":"Ingreso medio","value":"MedInc"},
                            {"label":"Edad media","value":"HouseAge"},
                            {"label":"Promedio de habitaciones","value":"AveRooms"},
                            {"label":"Promedio de dormitorios","value":"AveBedrms"},
                            {"label":"Población","value":"Population"},
                            {"label":"Promedio de ocupación","value":"AveOccuption"}
                        ],
                        value="MedHouseVal",
                        multi=False,
                        clearable=False)]),
            dcc.Graph(id="graph",className="e5_graph",figure={})
])

@app.callback(
    Output(component_id="graph",component_property="figure"),
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    
    scatter_map = px.scatter(df,x="Longitude",y="Latitude",color=slct_var)
    
    return scatter_map
    
if __name__ == "__main__":
    app.run_server(debug=False)

#### Aprendizaje no supervisado
las observaciones no tienen una respuesta asociada que guíe el aprendizaje, uno de sus algoritmos es Kmeans y genera. mediante un proceso iterativo, sus propias etiquetas determinando grupos de datos asociables en función de sus acercamientos estadísticos.

In [ ]:
clusters = []
inertias = []

for c in range(3,12):
    kmeans = KMeans(n_clusters=c).fit(df["MedHouseVal"].values.reshape((-1,1)))
    clusters.append(c)
    inertias.append(kmeans.inertia_)
    
kmeans = KMeans(n_clusters=5).fit(df["MedHouseVal"].values.reshape((-1,1)))
inertia = kmeans.inertia_

plt.plot(clusters, inertias, marker="o")
plt.text(int(str(kmeans)[-2]) - 0.1, inertia, "Valor del codo")
plt.grid("on")
plt.show()

#### Método del codo
El modelo de clustering requiere de un hiperparámetro que es la cantidad de centroides o K-means, estos serán dispersados e irán apropiandose de los datos más cercanos hasta convertirse en las medias de los grupos que formaron(nuevamente, mediante pruebas iterativas), el método del codo se utiliza a la hora de designar este valor, dónde el objetivo es visualizar la menor cantidad de centroides y la menor inercia(alejamiento entre los miembros de clusters y su centroide). También existen métricas especializadas en evaluar esta clase de modelos, Coeficiente de silueta e Índice Davies-Bouldin.

In [ ]:
kmeans = KMeans(n_clusters=5).fit(df["MedHouseVal"].values.reshape((-1,1)))

clusters = kmeans.labels_

df["clusters"] = clusters

range_values = np.array([])

for c in df["clusters"].sort_values().unique():
    cluster = df.loc[df["clusters"] == c,["clusters","MedHouseVal"]]
    max_value = str(cluster["MedHouseVal"].max())
    min_value = str(cluster["MedHouseVal"].min())
    range_values = np.append(range_values,min_value)
    range_values = np.append(range_values,max_value)
    
range_values = range_values.reshape((-1,2))
    
df["clusters"] = df["clusters"].replace(
    {
        0:f"0 ({range_values[0,0][:8]}$-{range_values[0,1][:8]}$)",
        1:f"1 ({range_values[1,0][:8]}$-{range_values[1,1][:8]}$)",
        2:f"2 ({range_values[2,0][:8]}$-{range_values[2,1][:8]}$)",
        3:f"3 ({range_values[3,0][:8]}$-{range_values[3,1][:8]}$)",
        4:f"4 ({range_values[4,0][:8]}$-{range_values[4,1][:8]}$)"
    })


fig = px.scatter(df,x="Longitude",y="Latitude",color="clusters")
fig.update_layout(title="Range of Houses'values")
fig 

##### Análisis de clusters
Ya tenemos una idea clara en cuanto a los rangos de precios que las viviendas suelen frecuentar y como su ubicación influye, por ejemplo: vemos un evindente aumento en las viviendas más cercanas a la costa de California, esto ubicando los cluster donde dentro de estos se encuentran los valores más caros siendo (296.800-414.100) y (414.300-500.000), también, observamos que los precios suelen valer entre 15.000 y 127.800 gracias al cluster que guarda esos valores y siendo el más numeroso